<a href="https://colab.research.google.com/github/ishaqbreiwish/football-match-ml-predictor/blob/main/Football_Match_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

<ipython-input-8-391eeaaee4f3>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [9]:
soup = BeautifulSoup(data.text) #new beautiful soup object
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]  # gets the link to the shooting tab

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0] #retrieves shooting data

<ipython-input-11-fbc35d1b9b7f>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0] #retrieves shooting data


In [12]:
shooting.head()

For Liverpool                                                              \
           Date   Time            Comp         Round  Day Venue Result   GF   
0    2024-08-17  12:30  Premier League   Matchweek 1  Sat  Away      W  2.0   
1    2024-08-25  16:30  Premier League   Matchweek 2  Sun  Home      W  2.0   
2    2024-09-01  16:00  Premier League   Matchweek 3  Sun  Away      W  3.0   
3    2024-09-14  15:00  Premier League   Matchweek 4  Sat  Home      L  0.0   
4    2024-09-17  21:00    Champions Lg  League phase  Tue  Away      W  3.0   

                         ... Standard               Expected               \
    GA         Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh   
0  0.0     Ipswich Town  ...     14.8  0.0  0     0      2.6  2.6    0.15   
1  0.0        Brentford  ...     13.6  1.0  0     0      2.5  2.5    0.14   
2  0.0   Manchester Utd  ...     13.4  0.0  0     0      1.8  1.8    0.16   
3  1.0  Nott'ham Forest  ...     14.9  0.0  0     0      0.9  0.9    0.07   
4  1.0         it Milan  ...     15.7  1.0  0     0      3.1  3.1    0.14   

               Unnamed: 25_level_0  
  G-xG np:G-xG        Match Report  
0 -0.6    -0.6        Match Report  
1 -0.5    -0.5        Match Report  
2  1.2     1.2        Match Report  
3 -0.9    -0.9        Match Report  
4 -0.1    -0.1        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel() #removes multilevel indices; will drop top row in this instance

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,4-2-3-1,Tim Robinson,Match Report,NaN,18,5,14.8,0.0,0,0
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,4-4-2,Stuart Attwell,Match Report,NaN,19,8,13.6,1.0,0,0
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,4-2-3-1,Anthony Taylor,Match Report,NaN,11,3,13.4,0.0,0,0
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,4-2-3-1,Michael Oliver,Match Report,NaN,14,5,14.9,0.0,0,0
4,2024-09-17,21:00,Champions Lg,League phase,Tue,Away,W,3.0,1.0,it Milan,...,4-2-3-1,Espen Eskås,Match Report,NaN,23,11,15.7,1.0,0,0


In [16]:
years = list(range(2024, 2016, -1)) # years we want to collect
all_matches = []

In [17]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats" # initial year

In [18]:
import time
from io import StringIO

for year in years:
    data = requests.get(standings_url) # Sends an HTTP GET request to `standings_url` to fetch the HTML content of the page
    if data.status_code != 200:
        print(f"Failed to retrieve page: {standings_url}")
        continue

    soup = BeautifulSoup(data.text, 'html.parser') # Parses the HTML content using BeautifulSoup to create a soup object
    standings_table = soup.select('table.stats_table') # selects the first table on the page that is of class "stats_table"
    if not standings_table:
        print("No standings table found on the page.")
        continue

    standings_table = standings_table[0] # selects the first table on the page that is of class "stats_table"
    links = [l.get("href") for l in standings_table.find_all('a')] # Extracts all href attributes from <a> (anchor) tags in the table

    team_urls = [f"https://fbref.com{l}" for l in links if '/squads/' in l]  # gets every link that contains to squads to ensure its only teams, and makes realtive links into absolute links

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls: # iterates through all the collected team urls
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ") #fetches team name from url and cleans it
        data = requests.get(team_url)  # sends http GET request to get team url

        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0] # gets scores & fixtures table

        soup = BeautifulSoup(data.text)  # parses html content to create soup object out of team page
        links = [l.get("href") for l in soup.find_all('a')]  # finding all 'a' tags to fetch href links
        links = [l for l in links if l and 'all_comps/shooting/' in l] # gets shooting stats

        data = requests.get(f"https://fbref.com{links[0]}")  # sends http GET request to fetch

        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0] #extract a specific HTML table from the data.text (the HTML content of a webpage) that matches the keyword "Shooting".
        shooting.columns = shooting.columns.droplevel() # drops multilevel index

        required_columns = ["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]
        try: #wrap in try block because not every team has shooting stats
            if not set(required_columns).issubset(shooting.columns):
              continue # skipping team
            team_data = matches.merge(shooting[required_columns], on="Date")
            team_data = team_data[team_data["Comp"] == "Premier League"]
            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
        except Exception as e:
          continue

        time.sleep(10)  # so you dont make the site slower by making a lot of requests at once

<ipython-input-18-a5a55aa3e1e8>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Season"] = year
<ipython-input-18-a5a55aa3e1e8>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Team"] = team_name
<ipython-input-18-a5a55aa3e1e8>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [19]:
len(all_matches)

160

In [21]:
match_df = pd.concat(all_matches)

In [22]:
match_df.columns = [c.lower() for c in match_df.columns]

In [23]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,Match Report,NaN,18.0,5.0,14.8,0.0,0,0,2024,Liverpool
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,Match Report,NaN,19.0,8.0,13.6,1.0,0,0,2024,Liverpool
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,Match Report,NaN,11.0,3.0,13.4,0.0,0,0,2024,Liverpool
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,Match Report,NaN,14.0,5.0,14.9,0.0,0,0,2024,Liverpool
5,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3.0,0.0,Bournemouth,...,Match Report,NaN,19.0,12.0,16.6,0.0,0,0,2024,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2018-04-15,16:00,Premier League,Matchweek 34,Sun,Away,W,1,0,Manchester Utd,...,Match Report,NaN,10.0,4.0,18.1,0.0,0,0,2017,West Bromwich Albion
39,2018-04-21,12:30,Premier League,Matchweek 35,Sat,Home,D,2,2,Liverpool,...,Match Report,NaN,13.0,6.0,17.7,0.0,0,0,2017,West Bromwich Albion
40,2018-04-28,15:00,Premier League,Matchweek 36,Sat,Away,W,1,0,Newcastle Utd,...,Match Report,NaN,9.0,2.0,20.1,0.0,0,0,2017,West Bromwich Albion
41,2018-05-05,15:00,Premier League,Matchweek 37,Sat,Home,W,1,0,Tottenham,...,Match Report,NaN,9.0,1.0,10.2,0.0,0,0,2017,West Bromwich Albion


In [24]:
match_df.to_csv("matches.csv")